# 20231114~15 장보경 코드
## 사용한 코드
- 기존 캐글 노트북 코드

## 공부내용
- 캐글 코드에 한국어 주석 처리 + 설명

The purpose of this notebook is to create a prediction model, which takes into account the metadata about patient and the samples as the ECG curves. Targets for the model will be superclasses as defined by the dataset.

Superclasses enumerated by dataset description are as follows:
```
Records | Superclass | Description
9528 | NORM | Normal ECG
5486 | MI | Myocardial Infarction
5250 | STTC | ST/T Change
4907 | CD | Conduction Disturbance
2655 | HYP | Hypertrophy
```

노트북의 주요 목적은 환자에 대한 메타데이터와 ECG 곡선 샘플을 고려하여 예측 모델을 생성하는 것입니다. 모델의 타겟은 데이터셋에 의해 정의된 상위 클래스(superclasses)가 될 것입니다.

데이터셋 설명에 의해 열거된 상위 클래스는 다음과 같습니다:

- 'NORM'은 정상 ECG를 나타내며, 9528개의 기록이 있습니다.
- 'MI'는 심근경색(Myocardial Infarction)을 나타내고, 5486개의 기록이 있습니다.
- 'STTC'는 ST/T 변화를 나타내며, 5250개의 기록이 있습니다.
- 'CD'는 전도 장애(Conduction Disturbance)를 나타내고, 4907개의 기록이 있습니다.
- 'HYP'는 비대증(Hypertrophy)을 나타내며, 2655개의 기록이 있습니다.

이 정보는 데이터셋의 구조와 분석 및 모델링할 때 중점을 둘 타겟 클래스를 이해하는 데 중요합니다. 예측 모델은 이러한 상위 클래스를 정확하게 분류할 수 있어야 합니다. 이 과정에서 ECG 데이터의 특징(feature) 추출, 전처리, 모델 선택, 훈련, 그리고 검증이 포함될 것입니다.


In [ ]:
import os
import ast
import wfdb

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow.keras as keras

sns.set_style('darkgrid')

First of all I need to load metadata about patients and samples provided by dataset. All metadata will be loaded to **ECG_df** and **SCP_df** dataframes respectively.

In [ ]:
# PTB-XL ECG 데이터셋의 경로를 지정합니다.
PATH_TO_DATA = '/kaggle/input/ptb-xl-dataset/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/'

# ECG 데이터를 CSV 파일에서 읽어옵니다. 'ecg_id' 컬럼을 인덱스로 사용합니다.
ECG_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'ptbxl_database.csv'), index_col='ecg_id')

# 'scp_codes' 컬럼의 값을 파이썬 객체로 변환합니다.
# 이 컬럼은 문자열 형태로 저장된 파이썬 표현을 실제 파이썬 객체로 변환하는데 사용됩니다.
ECG_df.scp_codes = ECG_df.scp_codes.apply(lambda x: ast.literal_eval(x))

# 다른 컬럼들의 데이터 타입을 변환합니다.
ECG_df.patient_id = ECG_df.patient_id.astype(int)   # 환자 ID를 정수형으로 변환
ECG_df.nurse = ECG_df.nurse.astype('Int64')         # 간호사 ID를 정수형으로 변환 (결측치를 고려한 변환)
ECG_df.site = ECG_df.site.astype('Int64')           # 측정 장소 ID를 정수형으로 변환 (결측치를 고려한 변환)
ECG_df.validated_by = ECG_df.validated_by.astype('Int64') # 검증자 ID를 정수형으로 변환 (결측치를 고려한 변환)

# SCP (Standardized Communication Protocol) 성명서 데이터를 로드합니다.
SCP_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'scp_statements.csv'), index_col=0)

# SCP 데이터 중에서 'diagnostic' 플래그가 1인 데이터만 필터링합니다.
# 이는 진단 목적으로 사용되는 성명서만을 고려하겠다는 의미입니다.
SCP_df = SCP_df[SCP_df.diagnostic == 1]

# ECG 데이터프레임을 출력합니다.
ECG_df

ECG samples are strattified to 10 groups. The authors of PTB-XL ECG dataset suggest use first 8 groups as the training samples. Last two groups then use as the validation and test sample set. 
I will accept this suggestion on my following work.

1. 샘플의 분할: PTB-XL ECG 데이터셋의 샘플들은 10개의 그룹으로 나뉩니다. 이러한 분할은 데이터셋의 저자들에 의해 제안되었습니다.

2. 학습과 검증/테스트 세트의 구분: 데이터셋 저자들은 처음 8개의 그룹을 학습 샘플로 사용할 것을 제안합니다. 그리고 나머지 두 개의 그룹은 검증 및 테스트 샘플 세트로 사용할 것을 권장합니다.

3. 방법론 수용: 이 마크다운 셀을 작성한 사용자는 PTB-XL ECG 데이터셋 저자들의 이러한 제안을 수용하여, 향후 작업에서 이 구분을 따르겠다고 밝히고 있습니다.샘플의 분할: PTB-XL ECG 데이터셋의 샘플들은 10개의 그룹으로 나뉩니다. 이러한 분할은 데이터셋의 저자들에 의해 제안되었습니다.

In [ ]:
ECG_df.strat_fold.value_counts()

I am going to add one more column **scp_classes** to ECG_df dataset, which represents all superlasses (as a list of abbreviations) assigned to the sample by cardiologists.

In [ ]:
# 진단 클래스를 추출하는 함수를 정의합니다.
# scp 파라미터는 SCP 코드의 딕셔너리를 받습니다.
def diagnostic_class(scp):
    res = set()  # 중복을 제거하기 위해 집합을 사용합니다.
    for k in scp.keys():
        if k in SCP_df.index:
            res.add(SCP_df.loc[k].diagnostic_class)
    return list(res)  # 집합을 리스트로 변환하여 반환합니다.
                    
# ECG 데이터프레임에 'scp_classes' 컬럼을 추가합니다.
# 각 ECG 샘플의 SCP 코드를 diagnostic_class 함수를 통해 처리합니다.
ECG_df['scp_classes'] = ECG_df.scp_codes.apply(diagnostic_class)

In [ ]:
ECG_df['scp_classes']

1. diagnostic_class 함수는 SCP 코드를 입력으로 받습니다. 이 함수는 각 SCP 코드에 대응하는 진단 클래스를 SCP_df 데이터프레임에서 찾아 이를 반환합니다. 중복을 제거하기 위해 set 자료구조를 사용합니다.

2. ECG_df 데이터프레임의 각 행에 대해 scp_codes 컬럼의 값을 diagnostic_class 함수에 적용합니다. 이 과정을 통해 각 ECG 샘플에 할당된 모든 진단 클래스를 추출합니다.

3. 추출한 진단 클래스를 ECG_df의 새로운 컬럼 scp_classes에 저장합니다. 이 컬럼은 각 샘플에 할당된 상위 클래스의 리스트를 포함하게 됩니다.

And finally I am going to load row data (ECG curves) to **ECG_data** dataset.

In [ ]:
def load_raw_data(df, sampling_rate, path):
    # 샘플링 레이트에 따라 다른 파일을 로드합니다.
    if sampling_rate == 100: # wfdb.rdsamp 함수는 신호 데이터와 메타데이터를 반환
        data = [wfdb.rdsamp(os.path.join(path, f)) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(os.path.join(path, f)) for f in df.filename_hr]
    
    # 로드된 데이터에서 신호 부분만을 추출합니다.
    data = np.array([signal for signal, meta in data])
    return data


sampling_rate = 100

ECG_data = load_raw_data(ECG_df, sampling_rate, PATH_TO_DATA)

ECG_data.shape

- sampling_rate를 100으로 설정합니다. 이는 데이터셋의 샘플링 레이트를 의미하며, ECG 데이터의 해상도를 결정합니다.
- load_raw_data 함수를 사용하여 ECG_df 데이터프레임에 대한 원시 ECG 신호 데이터를 로드합니다.
- 로드된 ECG_data의 형태(shape)를 확인합니다. 이는 데이터셋의 샘플 수와 각 샘플의 길이(시간 축의 길이)를 나타냅니다.

Now I have all data from **PTB-XL ECG dataset** loaded to Pandas dataframes or numpy arrays. So I can take a short look at them.

I do not want linger on data analysis for a long time. Very good job was already done by colleagues at [PTB XL Dataset Wrangling](https://www.kaggle.com/code/khyeh0719/ptb-xl-dataset-wrangling) notebook.
-> ANNIE가 분석한 캐글코드1임.

As an example I would like to show only one sample from ECG_data dataset:

In [ ]:
# ECG_data 배열의 첫 번째 샘플을 선택합니다.
# (이 샘플은 ECG 신호 데이터를 포함하고 있고, 여러 채널을 가질 수 있다.)
sample = ECG_data[0]


# 시각화를 위한 여러 개의 서브플롯을 생성합니다. 각 서브플롯은 ECG 채널 하나에 해당합니다.
# sample.shape[1]은 샘플의 채널 수를 나타냅니다.
bar, axes = plt.subplots(sample.shape[1], 1, figsize=(20,10))


# 각 ECG 채널에 대해 선 그래프를 그립니다.(x축은 샘플의 시간 축, y축은 신호의 강도)
for i in range(sample.shape[1]):
    sns.lineplot(x=np.arange(sample.shape[0]), y=sample[:, i], ax=axes[i])

# 주석 처리된 plt.tight_layout()은 서브플롯들이 서로 겹치지 않도록 조정합니다.
# plt.tight_layout()


plt.show()

위 코드 실행 결과를 보면, 12개 채널의 신호 데이터가 출력되는 듯 하다.

ECG 데이터의 각 채널은 심장의 다른 위치에서 측정한 신호를 나타내며, 이런 시각화를 통해 각 채널의 신호가 어떻게 다른지 알 수 있다.

First problem, I would like to cope with, are null values in metadata dataframe. There is a quick look at the problem:

missingno 라이브러리를 사용하여 ECG_df 데이터프레임에 있는 결측치(missing data)를 시각적으로 분석합니다. 이 매트릭스는 데이터프레임의 각 행과 열을 표시하며, 데이터가 있는 곳은 색이 칠해져 있고, 결측치가 있는 곳은 빈 공간으로 나타납니다.

In [ ]:
import missingno as msno

# ECG_df 데이터프레임의 결측치 분포를 시각화합니다.
msno.matrix(ECG_df)

# 그래프를 표시합니다.
plt.show()

And to add another angle of the view, there is an overview of unique values in all columns of metadata dataframe:

In [ ]:
#  'scp_codes'와 'scp_classes' 컬럼을 제외한 ECG_df의 모든 컬럼에 대해 몇 개의 고유값이 있는지
ECG_df[[col for col in ECG_df.columns if col not in ('scp_codes', 'scp_classes')]].nunique(dropna=True)
# dropna=True 옵션은 결측치(NaN 값)를 유니크한 값으로 고려하지 않음.

# Data preparation for modeling

I need first prepare input and output (targets) for my models. 

As inputs I will use both patient metadata (now loaded in the ECG_df dataframe) and ECG curves (in the ECG_data numpy array) respectively. But both require some rework to be useful for modeling, which will be done in following few steps.

As outputs I will create new dataframe with rows equal to samples and columns corresponding with diagnosis superclasses.

Because I will have two inputs and one output, I will preffix all created dataframes as follows:
- X - prefix for patient and sample metadata
- Y - prefix for ECG curves
- Z - prefix for targets

1. 입력 데이터 준비: 모델의 입력으로 사용될 데이터는 두 가지 유형입니다. 하나는 **환자의 메타데이터**로, 현재 ECG_df 데이터프레임에 로드되어 있습니다. 다른 하나는 **ECG 곡선 데이터**로, ECG_data 넘파이 배열에 저장되어 있습니다. 이 두 데이터 모두 모델링에 유용하게 사용하기 위해 일부 재작업이 필요합니다. 이 재작업은 다음 몇 단계에서 수행될 것입니다.

2. 출력(타겟) 데이터 준비: 타겟으로 사용될 새로운 데이터프레임을 생성할 예정입니다. 이 데이터프레임의 행은 샘플에 해당하며, 열은 진단 상위 클래스에 해당합니다.

3. 데이터프레임 접두사 규칙 설정
- X : 환자 및 샘플 메타데이터를 위해 사용됩니다. 이는 ECG_df 데이터프레임을 나타냅니다.
- Y : ECG 곡선 데이터를 위해 사용됩니다. 이는 ECG_data 넘파이 배열을 나타냅니다.
- Z : 타겟(진단 상위 클래스) 데이터를 위해 사용됩니다.

이 마크다운 셀은 데이터 전처리 및 모델링 과정을 위한 데이터의 구조화와 명확한 구분에 대한 계획을 제시합니다. 이러한 체계적인 접근 방식은 데이터 관리를 용이하게 하고, 모델링 과정에서 혼동을 방지하는 데 도움이 됩니다.

## X dataframe ...

I won't use all columns from ECG_df dataframe, but only a subset of them. 
Created dataframe **X** comprises only columns, witch are related to patient and his health condition. During dataframe creation I will cope with null values and mapping categorical columns to numerical representation.

- 사용자는 ECG_df 데이터프레임의 모든 컬럼을 사용하지 않고, 환자와 그의 건강 상태와 관련된 컬럼만을 포함하는 하위 집합을 사용할 계획입니다. 이렇게 함으로써 모델링에 더욱 관련성 높고 유용한 특성들만을 선택하게 됩니다.

- 새로 생성될 X 데이터프레임은 환자와 그의 건강 상태에 관련된 컬럼들만을 포함할 것입니다. 이는 환자의 메타데이터를 효과적으로 활용하기 위한 접근 방법입니다.

- 데이터프레임을 생성하는 동안에는 결측치(null values)를 처리하고, 범주형 컬럼을 수치적 표현으로 매핑하는 작업을 수행할 예정입니다. 결측치 처리는 데이터의 완전성과 정확도를 보장하는 중요한 단계이며, 범주형 컬럼을 수치화하는 것은 모델이 이해할 수 있는 형태로 데이터를 변환하는 중요한 과정입니다.

In [ ]:
# ECG_df의 인덱스를 사용하여 새로운 데이터프레임 X를 초기화합니다.
X = pd.DataFrame(index=ECG_df.index)


# age 컬럼을 ECG_df에서 복사하고, 결측치를 0으로
X['age'] = ECG_df.age
X.age.fillna(0, inplace=True)

# sex 컬럼을 ECG_df에서 복사하고, 수치형으로 변환한 후 결측치를 0으로
X['sex'] = ECG_df.sex.astype(float)
X.sex.fillna(0, inplace=True)

# height 컬럼을 복사하고, 50 미만의 값은 nan으로 설정. 결측치를 0으로
X['height'] = ECG_df.height
X.loc[X.height < 50, 'height'] = np.nan
X.height.fillna(0, inplace=True)

X['weight'] = ECG_df.weight
X.weight.fillna(0, inplace=True)


# 범주형에서 수치형으로 변환합니다. 'unknown'과 같은 비특정 값들은 0으로 매핑되며, 다른 값들은 순차적인 숫자로 매핑됩니다. 결측치도 0으로 채워집니다.
X['infarction_stadium1'] = ECG_df.infarction_stadium1.replace({
    'unknown': 0,
    'Stadium I': 1,
    'Stadium I-II': 2,
    'Stadium II': 3,
    'Stadium II-III': 4,
    'Stadium III': 5
}).fillna(0)

X['infarction_stadium2'] = ECG_df.infarction_stadium2.replace({
    'unknown': 0,
    'Stadium I': 1,
    'Stadium II': 2,
    'Stadium III': 3
}).fillna(0)

# 'ja, pacemaker'라는 값이 있으면 1로, 그렇지 않으면 0으로 변환
X['pacemaker'] = (ECG_df.pacemaker == 'ja, pacemaker').astype(float)

X

X 데이터프레임은 모델링에 사용될 환자의 메타데이터를 담고 있는데, 이 데이터는 결측치 처리와 필요한 변환을 거쳐 모델이 이해할 수 있는 수치형 데이터로 구성되어 있습니다.

## Y dataframe ...

At present I do not need to change data in the ECF_data dataframe, so I will use it as is.

## Z targets ...

I am going to create **Z** dataframe with columns corresponding to diagnoses superclasses.

In [ ]:
# Z 데이터프레임을 초기화
# 인덱스는 ECG_df의 인덱스를 사용
# 컬럼은 상위 클래스('NORM', 'MI', 'STTC', 'CD', 'HYP')로 구성
# 모든 값은 0으로 초기화되며, 데이터 타입은 정수형(int)

Z = pd.DataFrame(0, index=ECG_df.index, columns=['NORM', 'MI', 'STTC', 'CD', 'HYP'], dtype='int')


# 이중 for 루프를 사용하여 Z의 각 행(i)과 ECG_df에서 해당 행의 scp_classes 컬럼에 있는 각 클래스(k)에 대해 반복
# Z 데이터프레임에서 해당 행(i)과 컬럼(k)의 위치에 1을 할당
# 이 과정은 ECG_df의 각 샘플에 할당된 상위 클래스를 바이너리 형태(0 또는 1)로 변환
for i in Z.index:
    for k in ECG_df.loc[i].scp_classes:
        Z.loc[i, k] = 1

Z

Z 데이터프레임은 다중 레이블 분류 문제를 해결하기 위한 타겟 데이터로 사용될 수 있습니다. 각 샘플에 대해 다수의 진단 상위 클래스가 존재할 수 있으며, 이는 Z의 각 행에서 여러 개의 1이 나타날 수 있음을 의미합니다. 이 데이터프레임은 모델 학습 과정에서 타겟 변수로 활용됩니다.

## Splitting to train, validate and test datasets

As the authors of PTB-XL ECG dataset suggest, I will split all input and output dataset to training, validation and test subsets according *strat_fold* column.

In [ ]:
# ECG_df.strat_fold 컬럼의 값을 기준으로 데이터를 분할합니다. 
# 처음 8개 폴드는 학습용, 9번째 폴드는 검증용, 10번째 폴드는 테스트용으로 사용

X_train, Y_train, Z_train = X[ECG_df.strat_fold <= 8],  ECG_data[X[ECG_df.strat_fold <= 8].index - 1],  Z[ECG_df.strat_fold <= 8]
X_valid, Y_valid, Z_valid = X[ECG_df.strat_fold == 9],  ECG_data[X[ECG_df.strat_fold == 9].index - 1],  Z[ECG_df.strat_fold == 9]
X_test,  Y_test,  Z_test  = X[ECG_df.strat_fold == 10], ECG_data[X[ECG_df.strat_fold == 10].index - 1], Z[ECG_df.strat_fold == 10]


print(X_train.shape, Y_train.shape, Z_train.shape)
print(X_valid.shape, Y_valid.shape, Z_valid.shape)
print(X_test.shape,  Y_test.shape,  Z_test.shape)

(?) 위 코드에서 Y_train, Y_valid, Y_test를 구성할 때 ECG_data의 인덱스는 X 데이터프레임의 인덱스에서 1을 빼서 맞춥니다. 이는 ECG_data와 X, Z 데이터프레임 간의 인덱스 차이를 조정하기 위함입니다.

## Standardization of all input datasets

Wise people sometimes recommend to normalize/standardize data before using them for modeling and predictions. I will obey their opinions.

In [ ]:
from sklearn.preprocessing import StandardScaler

# StandardScaler 객체를 생성하고, X_train 데이터를 사용하여 스케일러를 학습시킵니다 (fit 메서드).
X_scaler = StandardScaler()
X_scaler.fit(X_train)

# transform 메서드를 사용하여 X_train, X_valid, X_test 데이터를 표준화합니다.
# 표준화된 데이터는 pandas.DataFrame으로 변환되어 원래 컬럼 이름을 유지
X_train = pd.DataFrame(X_scaler.transform(X_train), columns=X_train.columns)
X_valid = pd.DataFrame(X_scaler.transform(X_valid), columns=X_valid.columns)
X_test  = pd.DataFrame(X_scaler.transform(X_test),  columns=X_test.columns)

In [ ]:
# Y 데이터는 다차원 배열이므로, reshape(-1, Y_train.shape[-1])을 사용하여 2차원 배열로 변환합니다. 
# 이는 StandardScaler가 2차원 배열에 적합하게 동작하기 때문입니다.
Y_scaler = StandardScaler()
Y_scaler.fit(Y_train.reshape(-1, Y_train.shape[-1]))

# Y_scaler를 Y_train 데이터로 학습시키고, 이를 Y_valid와 Y_test에도 적용합니다.
# reshape(Y_train.shape)을 사용하여 원래의 3차원 형태로 다시 변환합니다.
Y_train = Y_scaler.transform(Y_train.reshape(-1, Y_train.shape[-1])).reshape(Y_train.shape)
Y_valid = Y_scaler.transform(Y_valid.reshape(-1, Y_valid.shape[-1])).reshape(Y_valid.shape)
Y_test  = Y_scaler.transform(Y_test.reshape(-1, Y_test.shape[-1])).reshape(Y_test.shape)

특히, 경사 하강법(Gradient Descent)과 같은 최적화 알고리즘을 사용하는 모델에서 표준화는 학습 속도와 성능을 크게 향상시킬 수 있습니다.

## Save all data to NPZ file

All previous steps took some time to complete. Loading all raw data is usually time consuming. So I decided to store all prepared data to file and use them later for my experiments with different models. 

In [ ]:
# .npz 확장자는 여러 넘파이 배열을 압축된 형태로 저장할 수 있는 파일 형식입니다.
NUMPY_DATA_FILE = '/kaggle/working/data.npz'

# 각 데이터 세트를 넘파이 배열로 변환하고, 데이터 타입을 float32로 설정합니다. 
# 이렇게 하면 파일 크기를 줄이고 메모리 사용을 최적화할 수 있습니다.
save_args = {
    'X_train': X_train.to_numpy().astype('float32'),
    'X_valid': X_valid.to_numpy().astype('float32'),
    'X_test':  X_test.to_numpy().astype('float32'),
    'Y_train': Y_train.astype('float32'), 
    'Y_valid': Y_valid.astype('float32'),
    'Y_test':  Y_test.astype('float32'),
    'Z_train': Z_train.to_numpy().astype('float32'), 
    'Z_valid': Z_valid.to_numpy().astype('float32'),
    'Z_test':  Z_test.to_numpy().astype('float32'),
}

# np.savez 함수를 사용하여 준비된 데이터를 지정된 파일 경로에 저장합니다. 
# (?) save_args 딕셔너리의 키는 파일 내부에서 각 데이터 배열의 이름으로 사용
np.savez(NUMPY_DATA_FILE, **save_args)

# 저장한 파일을 삭제
# os.remove(NUMPY_DATA_FILE)

 전처리된 데이터를 파일로 저장함으로써, 다양한 모델을 실험할 때마다 데이터를 처음부터 로드하고 전처리하는 시간을 절약할 수 있습니다.

# Models for metadata and row ECG curves

Targets for all models will be diagnosis superclasses assigned to every sample. Due to the fact, that a single sample can be marked by one o more superclasses, I will look for a **multilabel classification**.
For evaluation how succesful I am with my models, I will use preferably **binary accuracy** metric.

- 모델의 타겟은 각 샘플에 할당된 진단 상위 클래스(superclasses)입니다. 한 샘플에 하나 이상의 상위 클래스가 표시될 수 있기 때문에, 다중 레이블 분류(multilabel classification) 문제를 해결해야 합니다.
- 모델의 성공을 평가하기 위해 주로 이진 정확도(binary accuracy) 메트릭을 사용할 계획입니다. 이진 정확도는 각 레이블에 대해 모델이 얼마나 정확하게 예측했는지를 나타내는 지표입니다.

In [ ]:
import sys
import os
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import tensorflow.keras as keras

# TensorFlow가 출력하는 로그의 양을 제어합니다. 
#여기서 '3'은 정보, 경고, 그리고 오류 메시지를 제외한 모든 로그를 숨겨 출력을 깔끔하게 유지
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

sns.set_style('darkgrid')

I can read all prepared data at previous steps from NPZ file. This is useful, when I want to spare some work already done.

In [ ]:
NUMPY_DATA_FILE = '/kaggle/working/data.npz'

# sys.modules는 현재 로드된 모든 모듈을 담고 있는 딕셔너리입니다. 
# __name__은 현재 스크립트의 모듈 이름을 나타내므로, sys.modules[__name__]는 현재 실행 중인 스크립트의 모듈 객체를 참조합니다.
thismodule = sys.modules[__name__]

with np.load(NUMPY_DATA_FILE) as data:
    for k in data.keys():
        
        # 현재 모듈(thismodule)에 새 속성을 추가합니다. 
        # 속성의 이름은 k이며, 값은 data[k].astype(float)
        setattr(thismodule, k, data[k].astype(float))

위 코드는 저장된 데이터 파일로부터 다양한 데이터 세트(X_train, X_valid, X_test, Y_train 등)를 로드하고, 이를 현재 스크립트의 글로벌 변수로 설정하는 데 사용됩니다. 이렇게 하면 파일로부터 데이터를 로드한 후에도 마치 처음부터 변수에 저장된 것처럼 편리하게 데이터를 사용할 수 있습니다.

## Reference Model Baseline

Before I dive into real modeling, I would like to make some naive models as a reference for later comparison.

First naive model I try to evaluate: all samples have assigned diagnosis superclass "NORM" due to this class is most often represented.

In [ ]:
# Z_test와 동일한 형태를 가진 모두 0인 배열을 생성
z1 = np.zeros(Z_test.shape)

# 모든 샘플의 첫 번째 클래스('NORM')에 1을 할당합니다. 
# 이는 모든 샘플이 'NORM' 클래스에 속한다고 가정하는 것을 의미
z1[:, 0] = 1

# 이진 정확도 메트릭의 인스턴스를 생성
m = keras.metrics.BinaryAccuracy()
#실제 타겟 레이블(Z_test)과 예측 레이블(z1)을 비교
m.update_state(Z_test, z1)
m.result()

And the second reference model: all samples have assigned superlasses according to the probability of their representation in the train subset.

In [ ]:
# 각 클래스별로 Z_train에서 1의 개수를 합산합니다.
# 이를 Z_train의 총 행 수로 나누어 각 클래스가 나타나는 상대적인 빈도(확률)를 계산
z_prob = Z_train.sum(axis=0) / Z_train.shape[0]
# Z_test와 동일한 형태의 무작위 값 배열을 생성합니다.
z2 = np.random.uniform(size=Z_test.shape)

# 생성된 무작위 값이 해당 클래스의 존재 확률보다 낮은 경우에만 1로 설정합니다. 
# 이는 클래스의 존재 확률을 반영하여 무작위로 레이블을 할당하는 방식
for i in range(z2.shape[-1]):
    z2[:, i] = (z2[:, i] < z_prob[i]).astype('float64')
    
m = keras.metrics.BinaryAccuracy()
m.update_state(Z_test, z2)
m.result()

이 모델의 성능은 데이터에 존재하는 클래스의 불균형을 어느 정도 반영합니다. 예를 들어, 어떤 클래스가 매우 흔하다면(높은 존재 확률을 가진다면), 이 모델은 그 클래스를 더 자주 예측할 것입니다. 반면, 드문 클래스는 덜 자주 예측될 것입니다. 이러한 방식은 실제 레이블 분포와 어느 정도 일치하는 무작위 예측을 생성합니다.

# 딥러닝 모델 정의

## X Classifier 

This is the first model, I would like to evalute. I try to make prediction based on patient and sample metadata - X dataset.

The model is quite simple. It is composed of two fully connected Dense layers supplemented by Dropoout regularization layers. I will use this model on my later work, so it will be wise to prepare a function for it:

In [ ]:
def create_X_model(X, *, units=32, dropouts=0.3):
    
    X = keras.layers.Dense(units, activation='relu', name='X_dense_1')(X)
    X = keras.layers.Dropout(dropouts, name='X_drop_1')(X)
    X = keras.layers.Dense(units, activation='relu', name='X_dense_2')(X)
    X = keras.layers.Dropout(dropouts, name='X_drop_2')(X)
    
    return X

- 이 함수는 특정 수의 유닛(units)과 드롭아웃 비율(dropouts)을 가진 두 개의 밀집(Dense) 계층을 생성합니다.
- 각 밀집 계층 뒤에는 드롭아웃 계층이 배치되어 과적합을 방지하는 역할을 합니다.
- 활성화 함수로는 'relu'가 사용됩니다.

I would like to evaluate it, so I need complete model with additional Dense layers, regularization and final Dense layer with *sigmoid* activation function.

In [ ]:
def create_model01(X_shape, Z_shape):
    X_inputs = keras.Input(X_shape[1:], name='X_inputs')

    X = create_X_model(X_inputs)
    X = keras.layers.Dense(64, activation='relu', name='Z_dense_1')(X)
    X = keras.layers.Dense(64, activation='relu', name='Z_dense_2')(X)
    X = keras.layers.Dropout(0.5, name='Z_drop_1')(X)
    outputs = keras.layers.Dense(Z_shape[-1], activation='sigmoid', name='Z_outputs')(X)

    model = keras.Model(inputs=X_inputs, outputs=outputs, name='model01')
    return model

- 이 함수는 create_X_model을 사용하여 구성된 계층을 기반으로 최종 모델을 생성합니다.
- X_inputs는 모델의 입력 계층입니다.
- X_shape과 Z_shape는 입력 및 출력 데이터의 형태를 나타냅니다.
- 추가적으로 두 개의 밀집 계층과 드롭아웃 계층이 포함됩니다.
- 최종 출력 계층은 'sigmoid' 활성화 함수를 사용하는데 이는 다중 레이블 분류 문제에서 각 클래스에 대해 독립적인 확률을 출력하기 위함입니다. 'sigmoid' 활성화 함수는 각 출력 노드의 값이 0과 1 사이의 값으로 제한되며, 각각의 노드는 독립적인 이진 분류 문제처럼 작동합니다. 이는 각 클래스의 존재 여부를 개별적으로 예측하는 데 적합합니다.

### 모델 구조의 요약
- 입력 계층: 모델의 입력 데이터 형태(X_shape)에 따라 설정됩니다.
- 중간 계층: create_X_model 함수를 통해 생성된 두 개의 밀집 계층과 드롭아웃 계층을 포함합니다.
- 추가 밀집 계층: 두 개의 64 유닛 밀집 계층으로, 복잡한 패턴을 학습하는 데 도움을 줍니다.
- 드롭아웃 계층: 과적합을 방지하기 위해 추가된 드롭아웃 계층입니다.
- 출력 계층: Z_shape[-1]의 크기를 가진 밀집 계층으로, 각 클래스에 대한 예측 확률을 출력합니다. 'sigmoid' 활성화 함수 사용.

이 모델은 간단하지만, 환자 및 샘플 메타데이터를 기반으로 진단 상위 클래스를 예측하는 데 사용될 수 있는 기본적인 신경망 구조를 제공합니다. 이 모델은 기준 모델과 비교하여 성능 향상을 평가하는 데 사용될 수 있으며, 향후 다양한 하이퍼파라미터 조정 및 모델 구조 변경을 통해 더 개선될 수 있습니다.

Next few steps are stright forward. The **model01** is created, compiled and presented as a summary ...

In [ ]:
# 모델 생성
model01 = create_model01(X_train.shape, Z_train.shape)

# 모델 컴파일
model01.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy', 'Precision', 'Recall'])

# 모델 요약 출력
model01.summary()

1. 모델 생성:
- create_model01 함수를 호출하여 새 모델을 생성합니다. 이 함수는 X_train.shape과 Z_train.shape를 입력으로 받아 모델의 입력 및 출력 계층을 적절히 구성합니다.

2. 모델 컴파일:
- compile 메서드를 사용하여 모델을 컴파일합니다. 이 과정에서 세 가지 주요 요소를 설정합니다:
    - optimizer: 'adam'을 사용합니다. Adam 최적화 알고리즘은 효율적인 학습 속도 조절과 보편적으로 좋은 성능을 제공합니다.
    - loss: 'binary_crossentropy'를 손실 함수로 사용합니다. 이는 이진 분류 문제에 적합한 손실 함수입니다.
    - metrics: 모델의 성능을 평가하기 위해 'binary_accuracy', 'Precision', 'Recall'을 사용합니다. 이들 지표는 모델이 얼마나 잘 예측하는지를 다각도로 평가합니다.

3. 모델 요약 출력:
- summary 메서드를 호출하여 모델의 계층 구조, 출력 형태, 파라미터 수 등의 정보를 출력합니다. 이를 통해 모델의 전체적인 구조를 이해하고, 필요한 조정을 할 수 있습니다.

Model fitting will be stopped, when it ceases to improve. Then model with best results is read from checkpoint file.

In [ ]:
# 모델 체크포인트 경로 설정
MODEL_CHECKPOINT = '/kaggle/working/model/model01.keras'

# 콜백 리스트 생성
# patience=10은 10 에폭 동안 성능 개선이 없을 경우 훈련을 중단
# ModelCheckpoint: 검증 데이터셋의 성능이 개선될 때마다 모델을 저장합니다.
# save_best_only=True는 가장 좋은 모델만 저장한다는 것을 의미합니다.
callbacks_list = [
    keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=10),
    keras.callbacks.ModelCheckpoint(filepath=MODEL_CHECKPOINT, monitor='val_binary_accuracy', save_best_only=True)
]

# 모델 훈련
history = model01.fit(X_train, Z_train, epochs=40, batch_size=32, callbacks=callbacks_list, validation_data=(X_valid, Z_valid))

# 최적의 모델 로드
model01 = keras.models.load_model(MODEL_CHECKPOINT)

EarlyStopping 콜백은 과적합을 방지하고, ModelCheckpoint는 훈련 과정 중 가장 좋은 모델을 저장하여 나중에 재사용할 수 있게 해줍니다. 이렇게 함으로써, 모델 훈련 과정에서 자원을 효율적으로 사용하고, 최적의 성능을 가진 모델을 확보할 수 있습니다.

And finally you can see the fitting history with loss functions and metrics:

 history 객체에는 에폭(epoch)마다의 훈련 및 검증 데이터에 대한 손실(loss)과 정확도(accuracy), 정밀도(precision), 재현율(recall) 등의 메트릭 값이 저장되어 있습니다.

In [ ]:
sns.relplot(data=pd.DataFrame(history.history), kind='line', height=4, aspect=4)
plt.show()

I try to evaluate the model agains test dataset. As you can see, I can get better results than from naive models. So this is not the best result I can get, but it is useful.

In [ ]:
model01.evaluate(X_test, Z_test)

- X_test는 모델의 입력 데이터(환자 및 샘플 메타데이터)를 나타냅니다.
- Z_test는 실제 타겟 레이블(진단 상위 클래스)을 나타냅니다.
- evaluate 메서드는 지정된 테스트 데이터셋에 대해 모델의 손실과 정확도, 정밀도, 재현율 등의 메트릭을 계산하고 반환합니다.

## X and Y: 1D CNN Classifier

Now I am on the essention point of my work, 1D Convolution model for ECG curves improved by model for metadata.

First I need a model for curves. Once again it is a function, which creates three Conv1D layers supplemented by normalization and activation layers. Layers are separated by MaxPool1D layers, which reduce timeseries dimension of the input tensor. On the other side, Conv1D layers have growing number of filters, so the feature dimension is expanding. Whole model is finalized by GlobalAveragePooling1D layer (only features matter) and Dropout regularization.

And there is the function:

이 코드는 ECG 곡선 데이터를 처리하기 위한 1D 컨볼루션(1D Convolution) 모델을 생성하는 함수 create_Y_model을 정의합니다. 이 모델은 여러 컨볼루션 계층, 배치 정규화(batch normalization), 활성화 함수, 풀링(pooling) 계층 및 드롭아웃(dropout)을 포함합니다.

In [ ]:
# filters, kernel_size, strides를 매개변수로 받아, 각 컨볼루션 계층의 필터 수, 커널 크기, 스트라이드 값을 설정
def create_Y_model(X, *, filters=(32, 64, 128), kernel_size=(5, 3, 3), strides=(1, 1, 1)):
    
    f1, f2, f3 = filters
    k1, k2, k3 = kernel_size
    s1, s2, s3 = strides
    
    X = keras.layers.Conv1D(f1, k1, strides=s1, padding='same', name='Y_conv_1')(X)
    X = keras.layers.BatchNormalization(name='Y_norm_1')(X)
    X = keras.layers.ReLU(name='Y_relu_1')(X)

    X = keras.layers.MaxPool1D(2, name='Y_pool_1')(X)

    X = keras.layers.Conv1D(f2, k2, strides=s2, padding='same', name='Y_conv_2')(X)
    X = keras.layers.BatchNormalization(name='Y_norm_2')(X)
    X = keras.layers.ReLU(name='Y_relu_2')(X)

    X = keras.layers.MaxPool1D(2, name='Y_pool_2')(X)

    X = keras.layers.Conv1D(f3, k3, strides=s3, padding='same', name='Y_conv_3')(X)
    X = keras.layers.BatchNormalization(name='Y_norm_3')(X)
    X = keras.layers.ReLU(name='Y_relu_3')(X)

    X = keras.layers.GlobalAveragePooling1D(name='Y_aver')(X)
    X = keras.layers.Dropout(0.5, name='Y_drop')(X)

    return X

- 첫 번째 컨볼루션 계층(Conv1D)은 f1 필터와 k1 커널사이즈를 사용합니다. padding='same'은 입력과 출력의 크기를 동일하게 유지합니다.

- 배치 정규화(BatchNormalization)는 각 배치의 출력을 정규화하여 모델의 학습을 안정화시키는 역할을 합니다.
- ReLU 활성화 함수는 비선형성을 도입하여 모델이 더 복잡한 패턴을 학습할 수 있게 합니다.
- MaxPool1D 계층은 풀링(window) 크기 2를 사용하여 시계열 데이터의 차원을 줄입니다. 이는 특징을 압축하고 계산 효율성을 높이는데 도움을 줍니다.
- 다음 컨볼루션 계층은 필터 수(f2, f3), 커널 크기(k2, k3), 스트라이드(s2, s3)를 증가시켜, 모델이 더 세부적인 특징을 추출할 수 있게 합니다. 각 계층 후에도 배치 정규화와 ReLU 활성화 함수가 적용됩니다.
- 마지막으로, GlobalAveragePooling1D 계층은 각 컨볼루션 필터의 출력에 대한 평균을 계산하여, 특징의 전역적인 정보를 추출합니다.
- Dropout 계층은 과적합을 방지하기 위해 모델 학습 과정에서 일부 뉴런을 무작위로 비활성화합니다.

I will now proceed to combine models for ECG curves and patient metadata to one all encompassing model.

There are two imputs, X and Y datasets, and one output Z. I will concatenate models for metadata and ECG curves. The concatenation result will be fed to two fully connected Dense layers, followed by Dropout and final Dense layer with *sigmoid* activation. And that is all.

In [ ]:
def create_model02(X_shape, Y_shape, Z_shape):
    X_inputs = keras.Input(X_shape[1:], name='X_inputs')
    Y_inputs = keras.Input(Y_shape[1:], name='Y_inputs')

    X = keras.layers.Concatenate(name='Z_concat')([create_X_model(X_inputs), create_Y_model(Y_inputs, filters=(64, 128, 256), kernel_size=(7, 3, 3))])
    X = keras.layers.Dense(64, activation='relu', name='Z_dense_1')(X)
    X = keras.layers.Dense(64, activation='relu', name='Z_dense_2')(X)
    X = keras.layers.Dropout(0.5, name='Z_drop_1')(X)
    outputs = keras.layers.Dense(Z_shape[-1], activation='sigmoid', name='Z_outputs')(X)

    model = keras.Model(inputs=[X_inputs, Y_inputs], outputs=outputs, name='model02')
    return model

- 입력 계층: X_inputs는 환자 메타데이터, Y_inputs는 ECG 곡선 데이터를 위한 입력 계층입니다.
- 병합 계층: create_X_model과 create_Y_model을 통해 생성된 두 모델의 출력을 Concatenate 계층으로 병합합니다. 여기서 create_Y_model은 수정된 필터 크기와 커널 크기를 사용합니다.
- 추가 밀집 계층과 드롭아웃 계층: 이 계층들은 특징을 더 잘 학습하고 과적합을 방지하는 역할을 합니다.
- 출력 계층: Dense 계층은 Z_shape[-1]의 크기로 설정되며, 각 클래스에 대한 예측 확률을 출력합니다. 여기서도 'sigmoid' 활성화 함수가 사용됩니다.
- 모델 생성: keras.Model을 사용해 입력과 출력을 정의하고 모델을 생성합니다.

I can create model **model02**, compile it, and show its summary:

In [ ]:
model02 = create_model02(X_train.shape, Y_train.shape, Z_train.shape)
model02.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy', 'Precision', 'Recall'])
model02.summary()

1. 모델 생성
- create_model02 함수를 호출하여 model02 모델을 생성합니다. 이 함수는 환자의 메타데이터와 ECG 곡선 데이터를 처리할 수 있는 구조를 가진 모델을 만듭니다.

2. 모델 컴파일
- compile 메서드를 사용하여 모델을 컴파일합니다.
- optimizer: 'adam' 최적화 알고리즘을 사용합니다. Adam은 효율적인 학습률 조정을 제공합니다.
- loss: 이진 분류 문제에 적합한 'binary_crossentropy' 손실 함수를 사용합니다.
- metrics: 모델 성능을 평가하기 위해 'binary_accuracy', 'Precision', 'Recall' 지표를 사용합니다.

3. 모델 요약
- summary 메서드를 호출하여 모델의 구조를 출력합니다. 이를 통해 모델의 각 계층, 출력 형태, 파라미터 수 등을 확인할 수 있습니다.


- model02는 환자 메타데이터와 ECG 곡선 데이터를 통합적으로 처리하여 진단 상위 클래스를 예측하는 복잡한 구조의 모델입니다. 이 모델은 두 종류의 데이터에서 유의미한 정보를 추출하고 결합하여 정확도를 높이는 데 도움을 줄 수 있습니다.

Once again the model fitting is driven by early stopping and model checking callbacks. Following steps show fitting history and model evaluation against test dataset.

In [ ]:
MODEL_CHECKPOINT = '/kaggle/working/model/model02.keras'

callbacks_list = [
    keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=20),
    keras.callbacks.ModelCheckpoint(filepath=MODEL_CHECKPOINT, monitor='val_binary_accuracy', save_best_only=True)
]

history = model02.fit([X_train, Y_train], Z_train, epochs=100, batch_size=32, callbacks=callbacks_list, validation_data=([X_valid, Y_valid], Z_valid))

model02 = keras.models.load_model(MODEL_CHECKPOINT)

In [ ]:
sns.relplot(data=pd.DataFrame(history.history), kind='line', height=4, aspect=4)
plt.show()

In [ ]:
model02.evaluate([X_test, Y_test], Z_test)

## Final comments regarding models of X and Y

**I have achived 0.89 binary accuracy.**


What I have tried in the course of my work:

- change layers number and units count for the metadata model
- change layers number and units count for ECG curves model
- different pooling and regularizations for ECG curves model
- models based on ResNet architecture with respect to timeseries data, but I failed to achieve better results

- 메타데이터 모델과 ECG 곡선 모델에 대해 여러 가지 변화를 시도했습니다. 이러한 변화에는 계층 수와 유닛 수의 조정이 포함됩니다.
- ECG 곡선 모델에 다양한 풀링(pooling)과 정규화(regularization) 기법을 적용했습니다.
- 시계열 데이터에 맞춰진 ResNet 아키텍처를 기반으로 한 모델을 시도했지만, 더 나은 결과를 얻지 못했습니다.


# ECG signals augmentation during training

One of possibilities, how I could improve prediction results, is trying augmentation of ECG signals during training phase.
I hope that my model will be able to improve for significantly more epochs.

I will give it a try and will see ...

First of all I need augmented sources of training, validation and test data respectively. For this purpose I will prepare generator derived from Keras Sequence class.

What I would like to do in the generator:
- sliding window of ECG signals 
- noise added to ECG signals

- ECG 신호의 훈련 단계에서 데이터 증강을 시도하여 예측 결과를 개선할 수 있을 것으로 기대합니다. 이를 통해 모델이 더 많은 에폭(epoch)에 걸쳐 학습을 향상시킬 수 있을 것으로 예상합니다.

- 훈련, 검증, 테스트 데이터에 대한 증강된 소스를 준비하기 위해 Keras Sequence 클래스를 확장한 제너레이터를 만들 계획입니다.

If you look at the ECG signals then you see, that these are periodical signals with random beginning. So I try to randomly choose a fix-length window of sample signals - I call this "sliding window".

Generator has two parameters regarding sliding window:
- ```window_size``` - width of time-series window
- ```window_shift``` - if less than 0 then random shifting otherwise shifting by fixed value

Another augmentation of the ECG sample signals is adding a noise to all curves. I will use normal distribution with mean equal to zero and deviation equal to parameter ```sigma```.

(흥미로운 내용)
- 슬라이딩 윈도우: ECG 신호에 대해 고정 길이의 시계열 윈도우를 무작위로 선택합니다. 이를 '슬라이딩 윈도우'라고 합니다.
- 제너레이터는 슬라이딩 윈도우와 관련된 두 가지 매개변수를 가집니다:
    - window_size: 시계열 윈도우의 너비입니다.
    - window_shift: 0보다 작으면 무작위 이동을 의미하며, 그렇지 않으면 고정된 값으로 이동합니다.
- ECG 신호에 노이즈 추가: 모든 곡선에 정규 분포를 따르는 노이즈를 추가합니다. 노이즈는 평균이 0이고, 표준 편차가 sigma 매개변수 값과 같습니다.

이러한 데이터 증강 접근 방식은 ECG 신호의 다양성을 증가시키고, 모델이 더 일반화된 패턴을 학습하도록 도울 수 있습니다. 실제 ECG 신호는 무작위 시작점과 노이즈 등의 요소로 인해 다양성을 가질 수 있으므로, 이러한 증강 방법은 모델이 실제 환경에서 발생할 수 있는 다양한 신호 변형을 더 잘 이해하고 처리할 수 있게 만듭니다.

In [ ]:
# ECG 신호 데이터에 대한 슬라이딩 윈도우 함수 정의
def sliding_window(x, size, shift):
    # 주어진 size가 데이터 크기 범위 내에 있을 때
    if 0 < size < x.shape[0]:
        # shift가 음수이면 무작위로, 양수이면 지정된 값으로 윈도우 위치 결정
        shift = np.random.randint(0, x.shape[0] - size) if shift < 0 else shift
        # 슬라이딩 윈도우 적용된 데이터 반환
        return x[shift:size+shift, :]
    else:
        # size가 적절하지 않은 경우 원본 데이터 반환
        return x
    

# 데이터 증강을 위한 Keras Sequence 클래스를 확장한 제너레이터 클래스 정의
class AugmentedDataGenerator(keras.utils.Sequence):
    
    # 생성자: 데이터, 배치 크기, 윈도우 크기 및 이동량, 노이즈 정도(sigma) 설정
    def __init__(self, x, y, z, batch_size=32, window_size=0, window_shift=0, sigma=0.0, **kwargs):
        super(AugmentedDataGenerator, self).__init__(**kwargs)
        self.x = x  # 메타데이터
        self.y = y  # ECG 신호 데이터
        self.z = z  # 타겟 레이블
        self.batch_size = batch_size  # 배치 크기
        self.window_size = window_size  # 슬라이딩 윈도우 크기
        self.window_shift = window_shift  # 슬라이딩 윈도우 이동량
        self.sigma = sigma  # 노이즈 정도

    # x 데이터의 형태 반환
    @property
    def x_shape(self):
        return (self.batch_size, ) + self.x.shape[1:]

    # y 데이터의 형태 반환 (윈도우 크기에 따라 달라짐)
    @property
    def y_shape(self):
        return (self.batch_size, self.window_size if self.window_size > 0 else self.y.shape[1], ) + self.y.shape[2:]

    # z 데이터의 형태 반환
    @property
    def z_shape(self):
        return (self.batch_size, ) + self.z.shape[1:]

    # 제너레이터 길이 반환 (총 배치 수)
    def __len__(self):
        return math.ceil(len(self.y) / self.batch_size)

    # 인덱스에 해당하는 배치 데이터 생성 및 반환
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = np.array([sliding_window(r, self.window_size, self.window_shift) for r in self.y[idx * self.batch_size:(idx + 1) * self.batch_size]])
        batch_z = self.z[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        # 노이즈 추가 (sigma가 0보다 클 경우)
        if self.sigma > 0:
            batch_y += np.random.normal(loc=0.0, scale=self.sigma, size=batch_y.shape)
            
        return (batch_x, batch_y), batch_z

I will need further three instances of generator, namely for training, validation and test purpopses. 

Generator for training will provide 800 points sliding windows with random shifting of the window. In addition then adding noise to every ECG curve in the window.

Other two generators for validation and test purposes will provide 800 points sliding windows without shifting and noise.

In [ ]:
train_gen = AugmentedDataGenerator(X_train, Y_train, Z_train, window_size=800, window_shift=-1, sigma=0.05)
valid_gen = AugmentedDataGenerator(X_valid, Y_valid, Z_valid, window_size=800)
test_gen  = AugmentedDataGenerator(X_test, Y_test, Z_test, window_size=800)

1. 훈련 데이터 제너레이터 (train_gen):
- 800 포인트 길이의 슬라이딩 윈도우를 사용합니다.
- window_shift=-1은 윈도우의 위치를 무작위로 이동시킵니다.
- sigma=0.05는 ECG 곡선에 추가되는 노이즈의 정도를 설정합니다. 이 노이즈는 모델이 실제 세계의 다양한 조건에 대해 더 잘 일반화할 수 있도록 합니다.

2. 검증 및 테스트 데이터 제너레이터 (valid_gen, test_gen):
- 이들 제너레이터도 800 포인트 길이의 슬라이딩 윈도우를 사용합니다.
- window_shift와 sigma 매개변수를 설정하지 않았으므로, 윈도우 이동이나 노이즈 추가는 적용되지 않습니다. 검증 및 테스트 데이터는 증강되지 않고 원본 상태 그대로 사용됩니다.

이러한 설정은 훈련 데이터에 대해 다양한 변형을 적용하면서도 검증 및 테스트 데이터는 원본 상태로 유지하여 모델의 일반화 능력을 객관적으로 평가할 수 있게 합니다.

Here is a short look at one augmented sample:

In [ ]:
# 제너레이터의 첫 번째 배치를 가져옵니다.
(x, y), z = train_gen[0]
# y[0]은 이 배치의 첫 번째 ECG 샘플을 나타냅니다.
sample = y[0]
bar, axes = plt.subplots(sample.shape[1], 1, figsize=(20,10))
for i in range(sample.shape[1]):
    sns.lineplot(x=np.arange(sample.shape[0]), y=sample[:, i], ax=axes[i])
# plt.tight_layout()
plt.show()

Model is the same as before. I will use the same function ```create_model02``` for its instantiation.

In [ ]:
model03 = create_model02(train_gen.x_shape, train_gen.y_shape, train_gen.z_shape)
model03.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy', 'Precision', 'Recall'])
model03.summary()

And now I will try to fit the model to augmented training data:

In [ ]:
MODEL_CHECKPOINT = '/kaggle/working/model/model03.keras'

callbacks_list = [
    keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=20),
    keras.callbacks.ModelCheckpoint(filepath=MODEL_CHECKPOINT, monitor='val_binary_accuracy', save_best_only=True)
]

history = model03.fit(train_gen, epochs=100, batch_size=32, callbacks=callbacks_list, validation_data=valid_gen)

model03 = keras.models.load_model(MODEL_CHECKPOINT)

Finally a short look at the results:

In [ ]:
sns.relplot(data=pd.DataFrame(history.history), kind='line', height=4, aspect=4)
plt.show()

In [ ]:
model03.evaluate(test_gen)

## Final comment regarding data augmentation

As you can see, my results are not better than previous results without data augmentation. So I am going to try something else and hope I will have more luck.

# A deeper evaluation of all my previous models

For now I would like to take a deeper look at my previous models and result, when they are applicated on test datasets.

As usual I need to load all my preprocessed data, but only test dataset will be used for models evaluation.

In [ ]:
import sys
import os
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import tensorflow.keras as keras

import sklearn.metrics

sns.set_style('darkgrid')

In [ ]:
NUMPY_DATA_FILE = '/kaggle/working/data.npz'

thismodule = sys.modules[__name__]

with np.load(NUMPY_DATA_FILE) as data:
    for k in data.keys():
        setattr(thismodule, k, data[k].astype(float))

- NUMPY_DATA_FILE 변수에 저장된 데이터 파일 경로를 지정합니다.
- 현재 실행 중인 모듈(스크립트)에 대한 참조를 thismodule 변수에 저장합니다. 이는 sys.modules[__name__]를 사용하여 얻습니다.
- np.load 함수를 사용하여 .npz 파일을 열고, 파일 내의 데이터를 data 변수에 로드합니다.
- data.keys()를 순회하면서 각 키(k)에 해당하는 데이터를 현재 모듈의 속성으로 설정합니다. 이 때 setattr 함수를 사용하여 데이터를 현재 모듈의 속성으로 추가하고, 데이터 타입을 float로 변환합니다.

이 과정을 통해 저장된 데이터 파일로부터 필요한 테스트 데이터를 로드하여 이후의 모델 평가 과정에서 사용할 수 있게 됩니다. 데이터 로드 후에는 이전에 구축한 모델들을 이용하여 테스트 데이터셋에 대한 성능 평가를 수행할 수 있습니다.

This is auxiliary function for printing Confusion maps:

In [ ]:
# 혼동 행렬을 시각화하기 위한 함수 정의
def print_confusion_matrix(confusion_matrix, axes, class_label, class_names, fontsize=14):

    # 혼동 행렬을 판다스 데이터프레임으로 변환
    df_cm = pd.DataFrame(confusion_matrix, index=class_names, columns=class_names)

    # 혼동 행렬을 히트맵으로 시각화
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d", cbar=False, ax=axes)
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    
    # 히트맵의 y축과 x축 레이블 설정
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    
    # 축 레이블 및 제목 설정
    axes.set_ylabel('True label')
    axes.set_xlabel('Predicted label')
    axes.set_title("Class - " + class_label)

Now I am going to load all my models. They were saved during training phases as the ones with the best results. Subsequently I will predict results for test datasets.

In [ ]:
# 저장된 모델들을 로드합니다.
model01 = keras.models.load_model('/kaggle/working/model/model01.keras')
model02 = keras.models.load_model('/kaggle/working/model/model02.keras')
model03 = keras.models.load_model('/kaggle/working/model/model03.keras')

# 클래스 레이블을 정의합니다.
labels=['NORM', 'MI', 'STTC', 'CD', 'HYP']

# 테스트 데이터셋의 타겟 레이블을 정수형으로 변환합니다.
Z_test = Z_test.astype(int)

# 첫 번째 모델(model01)을 사용하여 테스트 데이터셋에 대한 예측을 수행합니다. 결과는 이진 형태로 변환합니다.
Z_pred_01 = model01.predict(X_test).round().astype(int)

# 두 번째 모델(model02)을 사용하여 테스트 데이터셋에 대한 예측을 수행합니다. 입력 데이터로 X_test와 Y_test를 사용합니다. 결과는 이진 형태로 변환합니다.
Z_pred_02 = model02.predict([X_test, Y_test]).round().astype(int)

# 세 번째 모델(model03)을 사용하여 테스트 데이터셋에 대한 예측을 수행합니다. 입력 데이터로 X_test와 Y_test의 첫 800개 포인트를 사용합니다. 결과는 이진 형태로 변환합니다.
Z_pred_03 = model03.predict([X_test, Y_test[:, :800, :]]).round().astype(int)

## Confusion Matrix and Classification Report for model01

And there is the detailed view of resuts for model01. You can see the confusion maps for all labels and the classification report.

This is the first model, which try to predict targets using metadata dataset only - X dataset. As you can see, I am able to predict only NORM and MI lables respectively. This is very poor result.

In [ ]:
fig, ax = plt.subplots(1, 5, figsize=(16, 3))
    
for axes, cfs_matrix, label in zip(ax.flatten(), sklearn.metrics.multilabel_confusion_matrix(Z_test, Z_pred_01), labels):
    print_confusion_matrix(cfs_matrix, axes, label, ["N", "Y"])
    
fig.tight_layout()
plt.show()

# 분류 성능 보고
# zero_division=0 옵션은 0으로 나누기 오류를 방지하기 위해 사용
print(sklearn.metrics.classification_report(Z_test, Z_pred_01, target_names=labels, zero_division=0))

## Confusion Matrix and Classification Report for model02

Second model uses metadata dataset X and ECG curves dataset Y. 

The results are better compared to model01. I am able to predict all labels. By assessing the metric Precission, I am able to predict all labels well. On the other side looking at the Recall metric, I am not successful for classifying labels CD and HYP respectively. This could be a problem.

In [ ]:
fig, ax = plt.subplots(1, 5, figsize=(16, 3))
    
for axes, cfs_matrix, label in zip(ax.flatten(), sklearn.metrics.multilabel_confusion_matrix(Z_test, Z_pred_02), labels):
    print_confusion_matrix(cfs_matrix, axes, label, ["N", "Y"])
    
fig.tight_layout()
plt.show()

print(sklearn.metrics.classification_report(Z_test, Z_pred_02, target_names=labels, zero_division=0))

## Confusion Matrix and Classification Report for model03

In the model03 I tried to use a data augmentation before using them for training model. I hoped this will help me to improve model02. 

As you can see I was not wery successful. The Precission metric looks a bit better, bud the Recall is noticeably worse. I consider the Recall metric more significant for this type of classification than Precision metric.

In [ ]:
fig, ax = plt.subplots(1, 5, figsize=(16, 3))
    
for axes, cfs_matrix, label in zip(ax.flatten(), sklearn.metrics.multilabel_confusion_matrix(Z_test, Z_pred_03), labels):
    print_confusion_matrix(cfs_matrix, axes, label, ["N", "Y"])
    
fig.tight_layout()
plt.show()

print(sklearn.metrics.classification_report(Z_test, Z_pred_03, target_names=labels, zero_division=0))